In [10]:
#to access the data/files from the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#path to the 'DataGenerator.py' and the 'DenseNet3D.py' files
%cd /content/drive/My Drive/gitRendu

#PATH variables (don't forget the last '/')
framesPATH = '/content/drive/My Drive/gitRendu/data/frames/' #where are stored the frames
masksPATH = '/content/drive/My Drive/gitRendu/data/masks/' #where are stored the masks
weightsPATH = '/content/drive/My Drive/gitRendu/data/weights/' #where will be saved the model

#training variables
NO_OF_TRAIN_IMAGES = 100 #don't give more training samples than available number of frames
BATCH_SIZE = 1 #batch size
nbrEpochs = 3 #nbr of training epoch
modelName = 'exampleModel' #the name of the model that will be saved
nbrClasses = 4 #total number of classes in the masks (counting '0' as a class)

/content/drive/My Drive/gitRendu


In [12]:
!pip install tensorflow;
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-a17twhck
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-a17twhck
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=ec996e58a5bf8d49bf43a29623d83f7e13ff300ec440973c733d943a5f454c0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-h5xa2uq1/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [13]:
%load_ext autoreload

%autoreload 2

import numpy as np
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from DataGenerator import DataGenerator
import DenseNet3D
from keras_contrib.applications import densenet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
%reload_ext autoreload

In [0]:
from keras import metrics
from DenseNet3D import DenseNet3D_FCN
#from DataGenerator import DataGenerator
import os
np.random.seed(1)

#we separate the dataset in training and validation sets
NO_OF_VALID_IMAGES = len(os.listdir(framesPATH)) - NO_OF_TRAIN_IMAGES
a = np.arange(len(os.listdir(framesPATH)))
np.random.shuffle(a)

train_ids = []
valid_ids = []

for i in range(NO_OF_TRAIN_IMAGES):
    train_ids.append(a[i])
    
for i in range(NO_OF_VALID_IMAGES):
    valid_ids.append(a[i+NO_OF_TRAIN_IMAGES])

partition = {}
    
partition['train'] = train_ids
partition['validation'] = valid_ids

In [0]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x, i.e. Ratio.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
    
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [17]:
from random import randint
Mask = []
N = len(os.listdir(masksPATH))
for ID in range(0,N): 
  mask = np.load(masksPATH+ 'mask_' + str(ID) + '.npy')
  Mask = np.append(Mask,mask)

weight = np.unique(Mask, return_counts = True)[1]
weight = np.sum(weight)/weight
print(weight)

[1.01018371e+00 6.64556910e+02 1.02412309e+03 1.31581764e+02]


In [18]:
print(N)

286


In [19]:
 # Parameters
params = {'dim': (80, 128, 32),
          'batch_size': 1,
          'n_classes': nbrClasses,
          'n_channels': 1,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'],framesPATH, masksPATH, **params)
validation_generator = DataGenerator(partition['validation'],framesPATH, masksPATH, **params)

#Augmentation
training_generator.augmentation_params(shift_range=0.1, rotate_range=15, zoom_range=0.1, augment=False, normalize=False, aug_prob = 0.6)
validation_generator.augmentation_params(shift_range=0.0, rotate_range=0, zoom_range=0.0, augment=False, normalize=False)

# Design model
model = DenseNet3D.DenseNet3D_FCN(input_shape=(80, 128, 32, 1),nb_dense_block=4, growth_rate=16,\
                       nb_layers_per_block=3, upsampling_type='upsampling', classes=params["n_classes"], activation='softmax')

opt = RMSprop(lr=0.001, rho=0.9)

checkpoint = ModelCheckpoint(weightsPATH, monitor='METRIC_TO_MONITOR', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

for i in range(params["n_classes"]):
  if i + 1 > len(weight):
    weight = np.append(weight, 0)

u_weight = weight.copy()
# u_weight[1:] = 0.5*weight[1:]
print(weight)
model.compile(loss = weighted_categorical_crossentropy(u_weight), optimizer = opt, metrics=['categorical_accuracy'])

callbacks_list = [csv_logger, checkpoint]

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,epochs= nbrEpochs, steps_per_epoch = (NO_OF_TRAIN_IMAGES//BATCH_SIZE),\
                    validation_steps=(NO_OF_VALID_IMAGES//BATCH_SIZE), callbacks = callbacks_list)

model.save(weightsPATH + modelName)






[1.01018371e+00 6.64556910e+02 1.02412309e+03 1.31581764e+02]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/3





100/100 [==============================] - 218s 2s/step - loss: 10.3501 - categorical_accuracy: 0.9306 - val_loss: 1.9645 - val_categorical_accuracy: 0.9758
Epoch 2/3


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with METRIC_TO_MONITOR available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


100/100 [==============================] - 122s 1s/step - loss: 1.4807 - categorical_accuracy: 0.9831 - val_loss: 1.2634 - val_categorical_accuracy: 0.9847
Epoch 3/3
100/100 [==============================] - 122s 1s/step - loss: 1.0018 - categorical_accuracy: 0.9882 - val_loss: 1.4240 - val_categorical_accuracy: 0.9812
